**BASIC MNIST EXAMPLE WITH MLP**

Imports...

In [14]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, Input, Reshape
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler as LRS


Define batch size, number of epochs and number of classes

In [2]:
batch_size = 100
epochs = 75
num_classes=10

Load MNIST and transform



In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

training set (60000, 28, 28)
test set (10000, 28, 28)


Define the NN topology, a sequential model with 2 hidden layers

In [4]:
model = Sequential()
model.add(Reshape(target_shape=(784,), input_shape=(784,)))
model.add(GN(0.3))

model.add(Dense(1024))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              803840    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 1024)              0         
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)           

2021-12-20 16:46:30.815047: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-20 16:46:30.815161: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Define an optimizer 

In [5]:
sgd=SGD(learning_rate=0.01, decay=1e-6, momentum=0.9)

Compile the model, define loss and link the optimizer

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

Finally, train the model and evaluate over the test data

In [15]:
def scheduler(epoch):
    if epoch < 25:
        return .1
    elif epoch < 50:
        return 0.01
    else:
        return 0.001

history = model.fit(
    x_train, 
    y_train,                
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test),
    callbacks=[LRS(scheduler)]
)

# Evaluate over test
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(f"Test error: {((1-score[1]) * 100):.3f}%")

Epoch 1/75
600/600 [==============================] - 8s 13ms/step - loss: 0.2712 - accuracy: 0.9307 - val_loss: 0.1104 - val_accuracy: 0.9710
Epoch 2/75
600/600 [==============================] - 8s 13ms/step - loss: 0.1446 - accuracy: 0.9581 - val_loss: 0.0728 - val_accuracy: 0.9792
Epoch 3/75
600/600 [==============================] - 8s 13ms/step - loss: 0.1138 - accuracy: 0.9654 - val_loss: 0.0614 - val_accuracy: 0.9811
Epoch 4/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0999 - accuracy: 0.9696 - val_loss: 0.0644 - val_accuracy: 0.9819
Epoch 5/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0902 - accuracy: 0.9722 - val_loss: 0.0566 - val_accuracy: 0.9835
Epoch 6/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0787 - accuracy: 0.9745 - val_loss: 0.0589 - val_accuracy: 0.9834
Epoch 7/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0721 - accuracy: 0.9771 - val_loss: 0.0519 - val_accuracy: 0.9836

Epoch 58/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0149 - accuracy: 0.9952 - val_loss: 0.0369 - val_accuracy: 0.9903
Epoch 59/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0146 - accuracy: 0.9950 - val_loss: 0.0369 - val_accuracy: 0.9900
Epoch 60/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0155 - accuracy: 0.9948 - val_loss: 0.0369 - val_accuracy: 0.9906
Epoch 61/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0154 - accuracy: 0.9951 - val_loss: 0.0371 - val_accuracy: 0.9902
Epoch 62/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0152 - accuracy: 0.9945 - val_loss: 0.0370 - val_accuracy: 0.9902
Epoch 63/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0160 - accuracy: 0.9944 - val_loss: 0.0370 - val_accuracy: 0.9902
Epoch 64/75
600/600 [==============================] - 8s 13ms/step - loss: 0.0138 - accuracy: 0.9952 - val_loss: 0.0375 - val_accuracy:

2021-12-20 17:06:29.163887: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Test loss: 0.037313785403966904
Test accuracy: 0.9904000759124756
Test error: 0.960%


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()